**Column Details**
- **PassengerId**: Unique identifier for each passenger, formatted as "Group_Number_Passenger".
- **HomePlanet**: The planet from which the passenger departed (e.g., Earth, Europa, Mars).
- **CryoSleep**: Indicates whether the passenger was in suspended animation during the voyage (True/False).
- **Cabin**: Passenger’s cabin number, structured as "Deck/Number/Side".
- **Destination**: The exoplanet destination of the passenger.
- **Age**: The passenger’s age in years.
- **VIP**: Whether the passenger had VIP status (True/False).
- **RoomService**: Amount spent on in-room service.
- **FoodCourt**: Amount spent at the food court.
- **ShoppingMall**: Amount spent in the shopping mall.
- **Spa**: Amount spent on spa services.
- **VRDeck**: Amount spent on the VR deck.
- **Name**: Full name of the passenger.
- **Transported**: Whether the passenger was transported to another dimension (True/False).
